## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [48]:
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
# Load up store_income_data.csv
store_data = 'store_income_data_task.csv'
df = pd.read_csv(store_data)

# Taking a look at the data set
df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [49]:
# Taking a look at all the unique values of the country column with .unique. It retured 77 unique countries
# countries = df['country'].unique()
# print(f"There are {len(countries)} unique countries")
# countries

# # Remove all non-alphabetic characters
# df['country'] = df['country'].str.replace(r'[^A-Za-z\s]', '', regex=True)
# Remove forward slashes
# df['country'] = df['country'].str.replace('/', '', regex=False)  
# Convert to lower case
df['country'] = df['country'].str.lower()
# Remove trailing white spaces
df['country'] = df['country'].str.strip()

# Taking a look at the now dataframe after the lower and str.strip manipulation. returning 37 unique countries
countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 37 unique countries


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [50]:
# This function, column_string_replacer is designed to replace strings within a specified column of a pandas DataFrame that are similar to a given target
def column_string_replacer(df, column, string_to_match, min_ratio=90):

    # Get the unique values from the specified column of the dataframe
    strings = df[column].unique()
    
    # Search for the string_to_match in these unique strings, using fuzzy matching
    # Limit the search to the top 10 closest matches
    # Use the token_sort_ratio scorer from fuzzywuzzy to sort tokens alphabetically 
    # and then join them back into a string - this helps in matching strings with jumbled parts
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    # Filter out the close matches which meet or exceed the minimum ratio specified
    # The ratio indicates how close the match should be - higher means more strictness
    # Here, only the string part of the match is kept
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]
    
    # Find all rows in the dataframe column that have a value matching the close match strings
    rows_with_matches = df[column].isin(close_matches)
    
    # Wherever a close match is found, replace the value in the dataframe with the intended string
    df.loc[rows_with_matches, column] = string_to_match
    # At this point, the function has replaced the closely matching strings
    # in the specified column with the desired string_to_match

In [51]:
# Using new fuction to replace strings within a specified column. That are similar to a given target
column_string_replacer(df=df, column='country', string_to_match="united kingdom")
column_string_replacer(df=df, column='country', string_to_match="uk")
column_string_replacer(df=df, column='country', string_to_match="britain")
column_string_replacer(df=df, column='country', string_to_match="england")
column_string_replacer(df=df, column='country', string_to_match="america")
column_string_replacer(df=df, column='country', string_to_match="united states")
column_string_replacer(df=df, column='country', string_to_match="united states of america")
column_string_replacer(df=df, column='country', string_to_match="africasouth africa")
column_string_replacer(df=df, column='country', string_to_match="sa")

countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries


There are 19 unique countries


array(['united states', 'britain', 'united kingdom', 'u.k.', 'sa', 'u.k/',
       'america', nan, 's.a.', 'england', 'uk', 's.a./', 'u.k', '',
       'united states of america', 's.a..', 'africasouth africa', '/',
       '.'], dtype=object)

In [59]:
# Remove periods
df['country'] = df['country'].str.replace('.', '', regex=False) 
# Remove forward slashes
df['country'] = df['country'].str.replace('/', '', regex=False)
# Turn null values to empty strings ('')
df['country'] = df['country'].fillna('')

# Convert the 'country' column to a pandas Series
series = df['country']

# Replace empty strings with 'Unknown'
series.replace('', 'Unknown', inplace=True)

# Checker
countries = series.unique()
countries = [country for country in countries if country]
print(f"There are {len(countries)} unique countries")
countries

There are 6 unique countries


['united states', 'united kingdom', 'uk', 'south africa', 'Unknown', 'sa']

In [60]:
# Replacing the rest of the data
df['country'] = df['country'].str.replace('united states of america', 'united states')
df['country'] = df['country'].str.replace('america', 'united states')
df['country'] = df['country'].str.replace('africasouth africa', 'south africa')
df['country'] = df['country'].str.replace('sa', 'south africa')
df['country'] = df['country'].str.replace('england', 'united kingdom')
df['country'] = df['country'].str.replace('britain', 'united kingdom')
df['country'] = df['country'].str.replace('uk', 'united kingdom')

# find out which country is most common
most_common_country = df['country'].mode()[0]
print(f"Most common country is {most_common_country}")
# Replacing null values with most common
df['country'] = df['country'].str.replace('Unknown', 'united kingdom')

# Checker
countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

Most common country is united kingdom
There are 3 unique countries


array(['united states', 'united kingdom', 'south africa'], dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [74]:
# Importing datetime
from datetime import datetime

# Check the data type of our date column
print(df['date_measured'].dtype)

# Convert 'date_measured' to datetime format with the correct format
df['date_measured'] = pd.to_datetime(df['date_measured'], format='%d-%m-%Y')

# Get the current date for reference
reference_date = datetime.today()
# Check date
# print(reference_date)

# Calculate the days ago and create the 'days_ago' column
df['days_ago'] = (reference_date - df['date_measured'])

# Checking new column days_ago
df['days_ago']

datetime64[ns]


0      6646 days 22:44:02.293391
1      6677 days 22:44:02.293391
2      7522 days 22:44:02.293391
3      6553 days 22:44:02.293391
4     18713 days 22:44:02.293391
                 ...            
995    6766 days 22:44:02.293391
996   12172 days 22:44:02.293391
997    5470 days 22:44:02.293391
998    4842 days 22:44:02.293391
999    4520 days 22:44:02.293391
Name: days_ago, Length: 1000, dtype: timedelta64[ns]